In [ ]:
#install needed module
%pip install spotipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 KB 6.1 MB/s eta 0:00:00


In [ ]:
import spotipy
import spotipy.util as util
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

## How To Set Up Authentication


1.   Make sure you have a Spotify for Developers Account: https://developer.spotify.com/

2. Create an App with Redirect URL "http://localhost:8888/callback"

3. Add yourself as a user in the App Settings with your spotify username and email

4. Set the username variable below to your Spotify username

5. Get the app client id and put it into the client_id variable below

6. Get the app secret id and put it into the client_secret variable below

7. Run the cell




In [ ]:
# Set up authentication with the Spotify API
username = 'jonathanesuola'
scope = 'user-library-read'
client_id = 'da486b61d5434f009d717e2252e44487'
client_secret = '903a2cda9f744b4fb7c7b7bf32aa4e99'
redirect_uri = 'http://localhost:8888/callback'

In [ ]:
from spotipy.oauth2 import SpotifyOAuth
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope, client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri, username=username, open_browser=False))

The following block of code will tell you to open a URL. Once you open it, copy the URL you were redirected to and paste it in the prompt box that will appear underneath the code block. Even if the redirection tells you there is an error, this will work.

In [ ]:
# Get the user's top tracks
#top_tracks = sp.current_user_top_tracks(limit=50, time_range='short_term')
top_tracks = sp.current_user_saved_tracks()

# Create a dataframe of track features
features = ['id', 'name', 'artists', 'popularity', 'danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
track_df = pd.DataFrame(columns=features)


SpotifyOauthError: ignored

In [ ]:
#print(top_tracks['items'][0]['track'])
track = top_tracks['items'][0]['track']
track_id = track['name']
print(track_id)

Broken Angel (feat. Helena)


In [ ]:
for i in range(len(top_tracks['items'])):
    track = top_tracks['items'][i]['track']
    track_id = track['id']
    track_name = track['name']
    track_artists = ', '.join([artist['name'] for artist in track['artists']])
    track_popularity = track['popularity']
    track_features = sp.audio_features(track_id)[0]
    track_df.loc[i] = [track_id, track_name, track_artists, track_popularity, track_features['danceability'], track_features['energy'], track_features['loudness'], track_features['speechiness'], track_features['acousticness'], track_features['instrumentalness'], track_features['liveness'], track_features['valence'], track_features['tempo']]

In [ ]:
track_df.head()

,id,name,artists,popularity,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,6vfQAHorSiUKMPvu0hBcI8,Broken Angel (feat. Helena),"Arash, Helena",53,0.654,0.802,-3.836,0.0251,0.2320,0.000002,0.0882,0.790,111.083
1,3lIeY717Kbj6FXVnUCmGhm,BoroBoro - Radio Edit,Arash,55,0.686,0.943,-3.181,0.0446,0.3270,0.002400,0.2750,0.843,94.984
2,3hxIUxnT27p5WcmjGUXNwx,Shut up My Moms Calling,Hotel Ugly,87,0.485,0.409,-10.711,0.1010,0.3250,0.000000,0.1030,0.376,138.419
3,0laYHRpNTS6i8FXdupHkJ4,Love You Like A Love Song,Selena Gomez & The Scene,82,0.858,0.678,-3.870,0.0469,0.0761,0.000000,0.0741,0.922,117.009
4,0AyA2lHB4YnQUNa8Y6NNjm,Sorry 4 What? // LV BELT,Tory Lanez,67,0.652,0.663,-8.096,0.0628,0.0199,0.000000,0.1320,0.280,125.803


In [ ]:
# Normalize the features
scaler = MinMaxScaler()
normalized_features = scaler.fit_transform(track_df[['popularity', 'danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']])

In [ ]:
# Compute cosine similarity between tracks
cosine_sim = cosine_similarity(normalized_features)

In [ ]:
# Create a neural network model
model = Sequential()
model.add(Dense(64, input_shape=(len(features)-3,), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(loss='mean_squared_error', optimizer='adam')

In [ ]:
# Train the model
X_train = normalized_features[:]
y_train = normalized_features[:,1]
model.fit(X_train, y_train, epochs=10, batch_size=50)

Epoch 1/10
1/1 [==============================] - 1s 1s/step - loss: 0.6206
Epoch 2/10
1/1 [==============================] - 0s 8ms/step - loss: 0.3670
Epoch 3/10
1/1 [==============================] - 0s 9ms/step - loss: 0.2894
Epoch 4/10
1/1 [==============================] - 0s 10ms/step - loss: 0.2637
Epoch 5/10
1/1 [==============================] - 0s 10ms/step - loss: 0.3326
Epoch 6/10
1/1 [==============================] - 0s 8ms/step - loss: 0.3146
Epoch 7/10
1/1 [==============================] - 0s 8ms/step - loss: 0.3039
Epoch 8/10
1/1 [==============================] - 0s 8ms/step - loss: 0.2022
Epoch 9/10
1/1 [==============================] - 0s 8ms/step - loss: 0.2607
Epoch 10/10
1/1 [==============================] - 0s 9ms/step - loss: 0.1749


In [ ]:
# get same info for top songs


In [ ]:
playlist_link = "https://open.spotify.com/playlist/5ABHKGoOzxkaa28ttQV9sE?si=UcFFHgJ1Rsm4YIT2LDBS4A"
playlist_URI = playlist_link.split("/")[-1].split("?")[0]
track_uris = [x["track"]["uri"] for x in sp.playlist_tracks(playlist_URI)["items"]]

In [ ]:
#Create arrays for different columns of dataset
track_uri = []
track_name = []
track_pop = []
artist_name= []
artist_uris = []
artist_pop = []
album_name = []
album_uri = []
for track in sp.playlist_tracks(playlist_URI)["items"]:
    #Track
    track_uri.append(track["track"]["uri"])
    track_name.append(track["track"]["name"])
    track_pop.append(track["track"]["popularity"])

    #Artist Info
    artist_name.append(track["track"]["artists"][0]["name"])
    artist_uri = track["track"]["artists"][0]["uri"]
    artist_uris.append(artist_uri)
    artist_info = sp.artist(artist_uri)
    artist_pop.append(artist_info["popularity"])
    # artist_info.append(sp.artist(artist_uri))

    #Album
    album_name.append(track["track"]["album"]["name"])
    album = track["track"]["album"]["uri"]

In [ ]:
df_choices = [calming, cheerful]


In [ ]:
dfTop100 = {'track_popularity': track_pop, 
            'track_name': track_name, 
            'track_id': track_id,
            'track_uri': track_uri, 
            'artist_name': artist_name, 
            'artist_popularity': artist_pop,
            'album_name': album_name,
            'popularity': track_popularity,
            }
dfTop100 = pd.DataFrame(data = dfTop100)
dfTop100.head()

,track_popularity,track_name,track_id,track_uri,artist_name,artist_popularity,album_name,popularity
0,91,Blinding Lights,0Z5HVNSXGXHf7MDb0zJHVW,spotify:track:0VjIjW4GlUZAMYd2vXMi3b,The Weeknd,98,After Hours,65
1,87,Shape of You,0Z5HVNSXGXHf7MDb0zJHVW,spotify:track:7qiZfU4dY1lWllzX7mPBI3,Ed Sheeran,92,÷ (Deluxe),65
2,78,Dance Monkey,0Z5HVNSXGXHf7MDb0zJHVW,spotify:track:2XU0oxnq2qxCpomAAuJY8K,Tones And I,72,Dance Monkey (Stripped Back) / Dance Monkey,65
3,88,Someone You Loved,0Z5HVNSXGXHf7MDb0zJHVW,spotify:track:7qEHsqek33rTcFNT9PFqLf,Lewis Capaldi,83,Divinely Uninspired To A Hellish Extent,65
4,0,rockstar,0Z5HVNSXGXHf7MDb0zJHVW,spotify:track:7wGoVu4Dady5GV0Sv4UIsx,Post Malone,89,rockstar,65


In [ ]:
#Add song characteristics information
def addAudioFeature(x, characteristic):
        return sp.audio_features(x)[0][characteristic]

In [ ]:
dfTop100['danceability']= dfTop100['track_id'].map(lambda x: addAudioFeature(x, 'danceability'))
dfTop100['acousticness']= dfTop100['track_id'].map(lambda x: addAudioFeature(x, 'acousticness'))
dfTop100['energy']= dfTop100['track_id'].map(lambda x: addAudioFeature(x, 'energy'))
dfTop100['instrumentalness']= dfTop100['track_id'].map(lambda x: addAudioFeature(x, 'instrumentalness'))
dfTop100['music_key']= dfTop100['track_id'].map(lambda x: addAudioFeature(x, 'key'))
dfTop100['liveness']= dfTop100['track_id'].map(lambda x: addAudioFeature(x, 'liveness'))
dfTop100['loudness']= dfTop100['track_id'].map(lambda x: addAudioFeature(x, 'loudness'))
dfTop100['mode']= dfTop100['track_id'].map(lambda x: addAudioFeature(x, 'mode'))
dfTop100['speechiness']= dfTop100['track_id'].map(lambda x: addAudioFeature(x, 'speechiness'))
dfTop100['tempo']= dfTop100['track_id'].map(lambda x: addAudioFeature(x, 'tempo'))
dfTop100['time_signature']= dfTop100['track_id'].map(lambda x: addAudioFeature(x, 'time_signature'))
dfTop100['valence']= dfTop100['track_id'].map(lambda x: addAudioFeature(x, 'valence'))
dfTop100['duration_ms']= dfTop100['track_id'].map(lambda x: addAudioFeature(x, 'duration_ms'))

In [ ]:
dfTop100.head()

,track_popularity,track_name,track_id,track_uri,artist_name,artist_popularity,album_name,popularity,danceability,acousticness,...,instrumentalness,music_key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,duration_ms
0,91,Blinding Lights,0Z5HVNSXGXHf7MDb0zJHVW,spotify:track:0VjIjW4GlUZAMYd2vXMi3b,The Weeknd,98,After Hours,65,0.947,0.156,...,0.000346,6,0.0651,-4.292,0,0.233,110.031,4,0.447,179800
1,87,Shape of You,0Z5HVNSXGXHf7MDb0zJHVW,spotify:track:7qiZfU4dY1lWllzX7mPBI3,Ed Sheeran,92,÷ (Deluxe),65,0.947,0.156,...,0.000346,6,0.0651,-4.292,0,0.233,110.031,4,0.447,179800
2,78,Dance Monkey,0Z5HVNSXGXHf7MDb0zJHVW,spotify:track:2XU0oxnq2qxCpomAAuJY8K,Tones And I,72,Dance Monkey (Stripped Back) / Dance Monkey,65,0.947,0.156,...,0.000346,6,0.0651,-4.292,0,0.233,110.031,4,0.447,179800
3,88,Someone You Loved,0Z5HVNSXGXHf7MDb0zJHVW,spotify:track:7qEHsqek33rTcFNT9PFqLf,Lewis Capaldi,83,Divinely Uninspired To A Hellish Extent,65,0.947,0.156,...,0.000346,6,0.0651,-4.292,0,0.233,110.031,4,0.447,179800
4,0,rockstar,0Z5HVNSXGXHf7MDb0zJHVW,spotify:track:7wGoVu4Dady5GV0Sv4UIsx,Post Malone,89,rockstar,65,0.947,0.156,...,0.000346,6,0.0651,-4.292,0,0.233,110.031,4,0.447,179800


In [ ]:
normalized_features2 = scaler.fit_transform(dfTop100[['popularity', 'danceability', 'energy', 'loudness', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']])

In [ ]:
X_test = normalized_features2[:]

In [ ]:
# Compute cosine similarity between tracks
cosine_sim2 = cosine_similarity(normalized_features2)

In [ ]:
# Predict popularity of tracks
predicted_popularity = model.predict(X_test)

# Combine the predicted popularity and cosine similarity into a recommendation score
recommendation_scores = predicted_popularity.flatten() * cosine_sim2[0]

# Get the top 10 recommended tracks
recommended_track_indices = recommendation_scores.argsort()[-10:][::-1]

# Get the top recommended track
recommended_track = recommendation_scores.argsort()[-1:][::-1]
print('Top track: ', dfTop100.iloc[recommended_track]['track_name'])

# Print the recommended tracks
print('Recommended tracks:')
for i in recommended_track_indices:
    print(dfTop100.iloc[i]['track_name'] + ' - ' + dfTop100.iloc[i]['artist_name'] + ' - ' + str(dfTop100.iloc[i]['popularity']))


4/4 [==============================] - 0s 2ms/step
Top track:  99    The Box
Name: track_name, dtype: object
Recommended tracks:
The Box - Roddy Ricch - 65
SAD! - XXXTENTACION - 65
Don't fall in love with me - Bolier Remix - Ana Shine - 65
Bohemian Rhapsody - Remastered 2011 - Queen - 65
Something Just Like This - The Chainsmokers - 65
Don't fall in love with me - Rawdolff Remix - Ana Shine Rawdolff - 65
Sweater Weather - The Neighbourhood - 65
Shallow - Lady Gaga - 65
As It Was - Harry Styles - 65
Love Yourself - Justin Bieber - 65
